In [32]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib.request

import numpy as np
import tensorflow as tf
import base64

# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

def main():
  # If the training and test sets aren't stored locally, download them.
  if not os.path.exists(IRIS_TRAINING):
    raw = urllib.request.urlopen(IRIS_TRAINING_URL).read().decode('utf-8')
    with open(IRIS_TRAINING, "w") as f:
      f.write(raw)

  if not os.path.exists(IRIS_TEST):
    raw = urllib.request.urlopen(IRIS_TEST_URL).read().decode('utf-8')
    with open(IRIS_TEST, "w") as f:
      f.write(raw)

  # Load datasets.
  training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TRAINING,
      target_dtype=np.int,
      features_dtype=np.float32)
  test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TEST,
      target_dtype=np.int,
      features_dtype=np.float32)

  # Specify that all features have real-value data
  feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

  # Build 3 layer DNN with 10, 20, 10 units respectively.
  classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                              hidden_units=[10, 20, 10],
                                              n_classes=3,
                                              model_dir="/tmp/iris_model")
  # Define the training inputs
  def get_train_inputs():
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)

    return x, y

  # Fit model.
  classifier.fit(input_fn=get_train_inputs, steps=2000)

  # Define the test inputs
  def get_test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)

    return x, y

  # Evaluate accuracy.
  accuracy_score = classifier.evaluate(input_fn=get_test_inputs,
                                       steps=1)["accuracy"]

  print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

  # Classify two new flower samples.
  def new_samples():
    return np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

  predictions = list(classifier.predict(input_fn=new_samples))

  print(
      "New Samples, Class Predictions:    {}\n"
      .format(predictions))

if __name__ == "__main__":
    main()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000026FD472D0F0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/iris_model'}
Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:loss = 1.5520251, step = 1
INFO:tensorflow:global_step/sec: 336.859
INFO:tensorflow:loss = 0.18421426, step = 101 (

In [1]:
import tensorflow as tf
import numpy as np
import urllib.request
import pandas as pd
import io
import os

IRIS_TRAIN = 'iris_training.csv'
IRIS_TEST = 'iris_test.csv'
IRIS_TRAIN_PATH = 'http://download.tensorflow.org/data/iris_training.csv'
IRIS_TEST_PATH = 'http://download.tensorflow.org/data/iris_test.csv'
NAMES =  ['SepLen', 'SepHei', 'PenLen', 'PenHei', 'Species']
BATCH_SIZE = 100
LEARNING_RATE = 0.5
STEPS = 1000

def get_data(is_get_train = True, batch_size = BATCH_SIZE):
    if is_get_train:
        filename = IRIS_TRAIN
        source = IRIS_TRAIN_PATH
    else:
        filename = IRIS_TEST
        source = IRIS_TEST_PATH

    if not os.path.exists(filename):
        raw = urllib.request.urlopen(source).read().decode('utf-8')
        with open(filename, 'w') as f:
            f.write(raw)
    data_pandas = pd.read_csv(filename, header = 0, names = NAMES, encoding = 'utf-8', engine = 'python')
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(data_pandas.iloc[:, :-1]), data_pandas.iloc[:, -1]))
    if is_get_train:
        dataset = dataset.shuffle(1000).repeat().batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()
    else:
        return dataset.batch(batch_size)
    
                       
get_data()


# ({'PenHei': <tf.Tensor 'IteratorGetNext_2:0' shape=(?,) dtype=float64>,
#   'PenLen': <tf.Tensor 'IteratorGetNext_2:1' shape=(?,) dtype=float64>,
#   'SepHei': <tf.Tensor 'IteratorGetNext_2:2' shape=(?,) dtype=float64>,
#   'SepLen': <tf.Tensor 'IteratorGetNext_2:3' shape=(?,) dtype=float64>},
#  <tf.Tensor 'IteratorGetNext_2:4' shape=(?,) dtype=int64>)

({'PenHei': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=float64>,
  'PenLen': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=float64>,
  'SepHei': <tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=float64>,
  'SepLen': <tf.Tensor 'IteratorGetNext:3' shape=(?,) dtype=float64>},
 <tf.Tensor 'IteratorGetNext:4' shape=(?,) dtype=int64>)

In [10]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()


pd_train = pd.read_csv(IRIS_TRAIN, header = 0, 
                       names = NAMES, 
                       encoding = 'utf-8', engine = 'python')

pd_test = pd.read_csv(IRIS_TEST, header = 0, 
                      names = NAMES, 
                      encoding = 'utf-8', engine = 'python')

train_X, train_y = pd_train, pd_train.pop('Species')
test_X, test_y = pd_test, pd_test.pop('Species')

In [20]:
my_feature_columns = []
for key in NAMES[:-1]:
    my_feature_columns.append(tf.feature_column.numeric_column(key = key))
my_feature_columns

[_NumericColumn(key='SepLen', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepHei', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PenLen', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PenHei', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [21]:
#Thiết lập params để truyền vào model function
my_params = {
    'feature_columns': my_feature_columns,
    'hidden_units': [10, 20 ,10],
    'n_classes': 3
}

#Khởi tạo model function
def my_model_fn(features, labels, mode, params):
    #Có 3 dạng mode chính trong model tương ứng với:
    #1. tf.estimator.ModeKeys.TRAIN: training model
    #2. tf.estimator.ModeKeys.PREDICT: dự báo model
    #3. tf.estimator.ModeKeys.EVAL: đánh giá model 
    #Bên dưới ta sẽ xây dựng cho mode TRAIN.
    
    #Khởi tạo mạng nơ ron với các 3 hidden layer:
    #1. Input layer: được khởi tạo từ tf.feature_column.input_layer(features, feature_columns).
    #features là các estimator của mạng nơ ron sẽ được map với Dataset thông qua tên trong feature_columns.
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    #2. Các hidden layers được khởi tạo bằng vòng lặp: Mỗi layer là một fully connected layer được khởi tạo
    #từ hàm tf.layers.dense(inputs, units, activation): 
    #trong đó inputs là một tensor (chính là các layer trước), units là số đơn vị của layer, activation là hàm kích hoạt.
    for units in params['hidden_units']:
         net = tf.layers.dense(net, units = units, activation = tf.nn.relu)
    #3. Output layer: Tính toán giá trị logits ứng với mỗi classes. Do đó ta cần một fully connected layers 
    #Kết nối mỗi layer trước đó với 3 units là 3 class.  Thông qua hàm kích hoạt softmax để tính xác xuất ước lượng.
    logits = tf.layers.dense(net, params['n_classes'], activation = tf.nn.softmax)
    
    #Hàm loss function: Đo lường giá trị entropy giữa phân phối dự báo từ từ layer logits và giá trị labels thực tế (ground truth)
    loss = tf.losses.sparse_softmax_cross_entropy(labels = labels, logits = logits)
    #Optimizer
    optimizer = tf.train.AdagradOptimizer(learning_rate = LEARNING_RATE)
    #Khởi tạo train operations tối thiểu hóa loss funtion
    train_op = optimizer.minimize(loss, global_step = tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode = tf.estimator.ModeKeys.TRAIN, loss = loss, train_op = train_op)


classifier = tf.estimator.Estimator(
    model_fn = my_model_fn,
    params = my_params #params sẽ được pass vào model_fn. 
    #Nội dung của params chính là các phần tử: feature_columns, units, n_class ta đã sử dụng trong my_mode_fn
)


# #train model. Bước này ta cần xác định hàm input_fn là gì, số bước huấn luyện.
classifier.train(
    input_fn = lambda:get_data(), #Lưu ý luôn phải để input_fn dưới dạng hàm lambda để có thể đưa dữ liệu vào huấn luyện theo iterator
    steps = STEPS
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LAPTOP~1\\AppData\\Local\\Temp\\tmpyldls7b1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001EB616C7710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 

In [14]:
train_input_fn(train_X, train_y, batch_size = BATCH_SIZE)

({'PenHei': <tf.Tensor 'IteratorGetNext_3:0' shape=(?,) dtype=float64>,
  'PenLen': <tf.Tensor 'IteratorGetNext_3:1' shape=(?,) dtype=float64>,
  'SepHei': <tf.Tensor 'IteratorGetNext_3:2' shape=(?,) dtype=float64>,
  'SepLen': <tf.Tensor 'IteratorGetNext_3:3' shape=(?,) dtype=float64>},
 <tf.Tensor 'IteratorGetNext_3:4' shape=(?,) dtype=int64>)